In [6]:
import csv
import os
import cv2
import numpy as np
from PIL import ImageFile
from keras_preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import tensorflow as tf
from keras import backend as K
from keras.applications import InceptionV3
from keras.models import Model
from keras import Sequential
from keras.activations import softmax
from keras.layers import Dense


def process_csv(csv_path):
    frame_dict = {}
    class_list = []
    with open(csv_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headers = next(csv_reader)  # Records the first row of csv

        for row in csv_reader:
            if row[4] != "":
                frame_dict[row[1]] = row[4]
                class_list.append(row[4])

    csv_file.close()
    return frame_dict, class_list





    

In [7]:
frame_dict, class_list = process_csv("../../labels/Frame Tagger - Singapore.csv")

# image_preprocessor("../../frames/Singapore Frames", frame_dict)

print(np.unique(class_list))

class_weights = class_weight.compute_class_weight('balanced', np.unique(class_list), class_list)

original_model = InceptionV3()
bottleneck_input = original_model.get_layer(index=0).input
bottleneck_output = original_model.get_layer(index=-2).output
bottleneck_model = Model(inputs=bottleneck_input, outputs=bottleneck_output)

for layer in bottleneck_model.layers:
    layer.trainable = False

new_model = Sequential()
new_model.add(bottleneck_model)
new_model.add(Dense(5, activation=softmax, input_dim=2048))

#sess = tf.Session()
tf.enable_eager_execution() 

def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

def weighted_accuracy(weights):
    #weights = K.variable(weights)
    
    def accuracy(y_true, y_pred):
        resIndices = K.argmax(y_pred, 1)
        testIndices = K.argmax(y_true, 1)
        # resIndices = resIndices
        # testIndices = testIndices
        
        # results = np.array([np.insert(np.zeros(4), index, 1) for index in resIndices])
        
        conf_matrix = confusion_matrix(resIndices, testIndices)
        
        true_pos = np.diag(conf_matrix)
        p = np.sum(conf_matrix, axis=0)
        r = np.sum(conf_matrix, axis=1)
        precision = [np.sum(true_pos[i] / p[i]) if p[i] > 0 else 0 for i in range(5)]
        recall = [np.sum(true_pos[i] / r[i]) if r[i] > 0 else 0 for i in range(5)]
        accuracy = np.sum(true_pos)/np.sum(conf_matrix)
        
        # if print_results:
        #     print(“Accuracy: {:.2%}“.format(accuracy))
        #     print(“Precision: {:.2%}“.format(np.mean(precision)))
        #     print(“Recall: {:.2%}“.format(np.mean(recall)))
        #     print(“Confusion Matrix:“)
        #     print(conf_matrix)
        return accuracy, precision, recall
        
        #with sess.as_default():
        # y_true = y_true.numpy()
        # y_pred = y_pred.numpy()
        # weight = weights
        # weight = weight / K.sum(weight, axis=-1, keepdims=True)
        # correct = np.ones_like(y_true)
        # correct[np.arange(len(y_true)), y_pred.argmax(axis=-1)] = 0
        # correct = np.sum(np.abs(y_true - correct) * weight, axis=-1)
        # accuracy = np.sum(correct) / len(y_true)
        # weight = weights
        # weight = weight / K.sum(weight, axis=-1, keepdims=True)
        # correct = K.ones_like(y_true)
        # #correct[K.arange(K.int_shape(y_true)[0]), y_pred.argmax(axis=-1)] = 0
        # correct = K.sum(K.abs(y_true - correct) * weight, axis=-1, keepdims=True)
        # accuracy = K.mean(correct)
        return K.constant(accuracy)
    return accuracy

# For a binary classification problem
new_model.compile(optimizer='rmsprop',
                  loss=weighted_categorical_crossentropy(class_weights),
                  metrics=['accuracy', weighted_accuracy(class_weights)])

datagen = ImageDataGenerator()
train_it = datagen.flow_from_directory("/Users/jd/Documents/Formula_1/data/train", target_size=(299, 299),
                                       batch_size=64, class_mode="categorical", color_mode="rgb")

test_it = datagen.flow_from_directory("/Users/jd/Documents/Formula_1/data/test", target_size=(299, 299),
                                      batch_size=64, class_mode="categorical", color_mode="rgb")

ImageFile.LOAD_TRUNCATED_IMAGES = True
new_model.fit_generator(train_it, class_weight=class_weights, epochs=1)

print(np.unique(class_list))
print(class_weights)


['isDriverView' 'isOtherView' 'isPitView' 'isSpectatorView' 'isTrackView']


AttributeError: module 'tensorflow' has no attribute 'enable_eager_execution'

In [ ]:

from sklearn.metrics import confusion_matrix, classification_report

print(new_model.metrics_names)
a = new_model.evaluate_generator(test_it)
print(a)

#Confution Matrix and Classification Report
Y_pred = new_model.predict_generator(test_it, 1326 // 64+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_it.classes, y_pred))
print('Classification Report')
target_names = np.unique(class_list)
print(classification_report(test_it.classes, y_pred, target_names=target_names))

Y_pred = new_model.predict_generator(train_it, 5721 // 64+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix _ Train')
print(confusion_matrix(train_it.classes, y_pred))



In [ ]:

tf.keras.model.save("CameraAngleModel.h5")


